In [3]:
import sys
sys.path.insert(0, "..")

import numpy as np
from matplotlib import pyplot as plt

from module.base.network import Network

from module.simulation.meanfield import MeanField
from module.simulation.quick_meanfield2 import QuickMeanField2
from module.simulation.meanfield2 import MeanField2
from module.simulation.set_meanfield2 import SetMeanField2

import module.components.CONST as CONST
from module.components.lawrence_dist import LawrenceDist

In [33]:
net = Network(5,5,1,[[0,0,0],[4,0,0],[0,4,0],[4,4,0]])
net.set_voltage_config([-0.01,0,0.01,0.02],0)

In [46]:
mf = MeanField(net)
mean = mf.numeric_integration_solve(mean, verbose = True)

convergence: 0.0006763633239512168


In [94]:
mf2 = QuickMeanField2(net)

In [98]:
mf2.solve(verbose = True, dt = 0.5)

convergence: (0.0003218750063750233, 9.594210250083034e-05)


In [99]:
np.where(np.array([0,1,2,3])== 1)

(array([1]),)